In [11]:
from opsml_artifacts import SnowflakeQueryRunner, ModelCard
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

### Get Sample Data

In [5]:
vocab_size = 500  # Only consider the top 20k words
maxlen = 100  # Only consider the first 200 words of each movie review
(x_train, y_train), (x_val, y_val) = keras.datasets.imdb.load_data(num_words=vocab_size)
print(len(x_train), "Training sequences")
print(len(x_val), "Validation sequences")
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_val = keras.preprocessing.sequence.pad_sequences(x_val, maxlen=maxlen)

25000 Training sequences
25000 Validation sequences


### Create Model

In [4]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

### Combine model  layers
- making model smaller for example

In [7]:
embed_dim = 4  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 4  # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(4, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(2, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

### Fit Model

In [10]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
history = model.fit(
    x_train.astype(np.float32), y_train, batch_size=64, epochs=1, steps_per_epoch=20,validation_data=(x_val.astype(np.float32), y_val)
)

20/20 [==============================] - 6s 248ms/step - loss: 0.6985 - accuracy: 0.4945 - val_loss: 0.6940 - val_accuracy: 0.4996


### Test ModelCard

In [13]:
MODEL_NAME = "transformer"
TEAM = "SPMS"
USER_EMAIL = "steven.forrester@shipt.com"

model_card = ModelCard(
    name=MODEL_NAME,
    team=TEAM,
    user_email=USER_EMAIL,
    trained_model=model,
    sample_input_data=x_train[0:1],
)

2023-01-24 02:08:02.711609: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-01-24 02:08:02.711832: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-01-24 02:08:02.899616: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-01-24 02:08:02.899848: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session


1/1 [==============================] - 0s 228ms/step
{"level": "INFO", "message": "Validating converted onnx model", "timestamp": "2023-01-24T02:08:05.978712Z", "app_env": "staging", "host": null, "version": null}
{"level": "INFO", "message": "Onnx model validated", "timestamp": "2023-01-24T02:08:06.017909Z", "app_env": "staging", "host": null, "version": null}


In [14]:
onnx_model = model_card.model()

{"level": "WARNING", "message": "ModelCard has no version (not registered).\n                Defaulting to 1 (for testing only)\n            ", "timestamp": "2023-01-24T02:08:24.329488Z", "app_env": "staging", "host": null, "version": null}


In [15]:
onnx_model.input_sig.schema()

{'title': 'Features',
 'type': 'object',
 'properties': {'input_2': {'title': 'Input 2',
   'minItems': 100,
   'maxItems': 100,
   'type': 'array',
   'items': {'type': 'number'}}},
 'required': ['input_2']}

In [16]:
onnx_model.output_sig.schema()

{'title': 'Features',
 'type': 'object',
 'properties': {'dense_7': {'title': 'Dense 7',
   'minItems': 2,
   'maxItems': 2,
   'type': 'array',
   'items': {'type': 'number'}}},
 'required': ['dense_7']}